# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup  
import hvplot.pandas # for maps          
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ranikhet,29.6500,79.4167,15.22,87,0,1.69,IN,1696007525
1,1,queenstown,-31.8976,26.8753,21.44,24,1,3.36,ZA,1696007525
2,2,macapa,0.0389,-51.0664,33.99,49,40,6.69,BR,1696007389
3,3,port-aux-francais,-49.3500,70.2167,1.33,73,25,19.71,TF,1696007525
4,4,chonchi,-42.6128,-73.8092,11.56,54,13,2.04,CL,1696007525


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot  
# NOTE: order of "Lat" and "Lng" matters
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,   # scale by default is 1
    color = "City",
    # "alpha "to control the transparency/opacity of the data points - specially useful when having lots of data --- 
    # useful for visualizing overlapping data points
    alpha = 0.5,   
    tiles = "OSM", 
    frame_width = 700,
    frame_height = 500,
    use_index = False, # don't want the indices to be shown when hovering over the data
    hover_cols= "all" 
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,City_ID,Max Temp,Cloudiness,Wind Speed,Country,Date)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit the following criteria:
# *    Max Temperature between 21 and 27 degrees,
# *    Wind speed less than 4.5 m/s,
# *    And I'd prefer having clear or some clouds sky, **
# *    and not too humid. **
# Then drop any results with null values 
ideal_weather_df = city_data_df.loc[
    (city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] <= 50) \
    & (city_data_df["Humidity"] <= 55)
]  

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna() # empty by default is "any"

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,queenstown,-31.8976,26.8753,21.44,24,1,3.36,ZA,1696007525
18,18,eruh,37.7517,42.1811,22.46,27,0,3.30,TR,1696007527
67,67,demopolis,32.5176,-87.8364,26.97,50,0,0.00,US,1696007345
100,100,mount darwin,-16.7725,31.5838,25.37,39,47,2.26,ZW,1696007533
110,110,delice,39.9537,34.0259,22.16,41,27,1.40,TR,1696007534
165,165,vryburg,-26.9566,24.7284,23.52,20,27,3.71,ZA,1696007541
203,203,mount isa,-20.7333,139.5000,25.87,24,0,3.09,AU,1696007544
304,304,prieska,-29.6641,22.7474,26.43,10,47,3.72,ZA,1696007552
338,338,marshall,34.3834,-86.3333,26.17,53,0,0.00,US,1696007556
414,414,mandal,25.4500,74.5667,26.71,34,29,1.78,IN,1696007564


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Then Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
1,queenstown,ZA,-31.8976,26.8753,24,
18,eruh,TR,37.7517,42.1811,27,
67,demopolis,US,32.5176,-87.8364,50,
100,mount darwin,ZW,-16.7725,31.5838,39,
110,delice,TR,39.9537,34.0259,41,
165,vryburg,ZA,-26.9566,24.7284,20,
203,mount isa,AU,-20.7333,139.5000,24,
304,prieska,ZA,-29.6641,22.7474,10,
338,marshall,US,34.3834,-86.3333,53,
414,mandal,IN,25.4500,74.5667,34,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000 
params = {
    "categories": "accommodation.hotel", 
    "apiKey":geoapify_key, 
    "limit": 20  # This is optional-would be good in case of large cities to prevent errors
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"] 
    lon = row["Lng"] 
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}" 
    params["bias"] = f"proximity:{lon},{lat}" 
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params) 
    
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json() 

    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
queenstown - nearest hotel: Queens Hotel
eruh - nearest hotel: No hotel found
demopolis - nearest hotel: No hotel found
mount darwin - nearest hotel: No hotel found
delice - nearest hotel: No hotel found
vryburg - nearest hotel: International Hotel
mount isa - nearest hotel: Ibis Styles
prieska - nearest hotel: No hotel found
marshall - nearest hotel: Wyndham Garden Lake-Guntersville
mandal - nearest hotel: No hotel found
glace bay - nearest hotel: No hotel found
farmington - nearest hotel: Casa Blanca Inn & Suites
gokwe - nearest hotel: No hotel found
monchique - nearest hotel: Abrigo da Montanha
tabas - nearest hotel: هتل امیر


,City,Country,Lat,Lng,Humidity,Hotel Name
1,queenstown,ZA,-31.8976,26.8753,24,Queens Hotel
18,eruh,TR,37.7517,42.1811,27,No hotel found
67,demopolis,US,32.5176,-87.8364,50,No hotel found
100,mount darwin,ZW,-16.7725,31.5838,39,No hotel found
110,delice,TR,39.9537,34.0259,41,No hotel found
165,vryburg,ZA,-26.9566,24.7284,20,International Hotel
203,mount isa,AU,-20.7333,139.5000,24,Ibis Styles
304,prieska,ZA,-29.6641,22.7474,10,No hotel found
338,marshall,US,34.3834,-86.3333,53,Wyndham Garden Lake-Guntersville
414,mandal,IN,25.4500,74.5667,34,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotels_map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM", 
    frame_width = 700,
    frame_height = 500,
    hover_cols= ["Hotel Name", "Country"] # or I could do "all" - but I want Country to be at the bottom

)

# Display the map
hotels_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)